# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
A. Gould  ->  A. Gould  |  ['A. Gould']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Feldt  ->  M. Feldt  |  ['M. Feldt']
H. Steuer  ->  H. Steuer  |  ['H. Steuer']
T. Bertram  ->  T. Bertram  |  ['T. Bertram']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
Arxiv has 51 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2311.12932


extracting tarball to tmp_2311.12932...

 done.
Retrieving document from  https://arxiv.org/e-print/2311.13097


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3548: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3548: LatexWarning: Found documentclass in tmp_2311.12932/cr_igimf.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

✔ → 9801:\section{Observations and data reduction}\label{sec:observations}
  ↳ 14777:\section{Analysis framework}\label{sec:methods}


✔ → 14777:\section{Analysis framework}\label{sec:methods}
  ↳ 28234:\section{Atmospheric models}\label{sec:atmosphere_models}


✔ → 28234:\section{Atmospheric models}\label{sec:atmosphere_models}
  ↳ 35994:\section{Results}\label{sec:results}
✔ → 35994:\section{Results}\label{sec:results}
  ↳ 43067:\section{Discussion}\label{sec:discussion}


✔ → 43067:\section{Discussion}\label{sec:discussion}
  ↳ 50366:\section{Summary and outlook}\label{sec:conclusions}
✔ → 50366:\section{Summary and outlook}\label{sec:conclusions}
  ↳ 52647:\section{Visual comparison with Snellen et al. 2014}\label{app:crires_comp}
✔ → 52647:\section{Visual comparison with Snellen et al. 2014}\label{app:crires_comp}
  ↳ 53193:\section{Full corner plot for the nominal model}\label{app:corner_plot}
✔ → 53193:\section{Full corner plot for the nominal model}\label{app:corner_plot}
  ↳ 53515:end


Found 60 bibliographic references in tmp_2311.13527/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.13097-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.13097) | **KMT-2023-BLG-1431Lb: A New $q < 10^{-4}$ Microlensing Planet from a  Subtle Signature**  |
|| A. Bell, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2023-11-23*|
|*Comments*| *PASP submitted. arXiv admin note: text overlap with arXiv:2301.06779*|
|**Abstract**| The current studies of microlensing planets are limited by small number statistics. Follow-up observations of high-magnification microlensing events can efficiently form a statistical planetary sample. Since 2020, the Korea Microlensing Telescope Network (KMTNet) and the Las Cumbres Observatory (LCO) global network have been conducting a follow-up program for high-magnification KMTNet events. Here, we report the detection and analysis of a microlensing planetary event, KMT-2023-BLG-1431, for which the subtle (0.05 magnitude) and short-lived (5 hours) planetary signature was characterized by the follow-up from KMTNet and LCO. A binary-lens single-source (2L1S) analysis reveals a planet/host mass ratio of $q = (0.72 \pm 0.07) \times 10^{-4}$, and the single-lens binary-source (1L2S) model is excluded by $\Delta\chi^2 = 80$. A Bayesian analysis using a Galactic model yields estimates of the host star mass of $M_{\rm host} = 0.57^{+0.33}_{-0.29}~M_\odot$, the planetary mass of $M_{\rm planet} = 13.5_{-6.8}^{+8.1}~M_{\oplus}$, and the lens distance of $D_{\rm L} = 6.9_{-1.7}^{+0.8}$ kpc. The projected planet-host separation of $a_\perp = 2.3_{-0.5}^{+0.5}$ au or $a_\perp = 3.2_{-0.8}^{+0.7}$, subject to the close/wide degeneracy. We also find that without the follow-up data, the survey-only data cannot break the degeneracy of central/resonant caustics and the degeneracy of 2L1S/1L2S models, showing the importance of follow-up observations for current microlensing surveys. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.13437-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.13437) | **Simulating METIS SCAO System**  |
|| <mark>M. Feldt</mark>, et al. -- incl., <mark>H. Steuer</mark>, <mark>T. Bertram</mark> |
|*Appeared on*| *2023-11-23*|
|*Comments*| *17 pages, 14 Figures, AO4ELT VII Conference - Avignon - June 2023*|
|**Abstract**| METIS, the Mid-Infrared ELT Imager and Spectrograph, is one of the four first-generation ELT instruments scheduled to see first light in 2028. Its two main science modules are supported by an adaptive optics system featuring a pyramid sensor with 90x90 subapertures working in the H and K bands. During the PDR and FDR phases, extensive simulations were carried out to support the sensing, reconstruction, and control concept of METIS single-conjugate adaptive optics (SCAO) system. We present details on the implementation of the COMPASS-based environment used for the simulations, the metrics used for analyzing our performance expectations, an overview of the main results, and some details on special cases like non-common path aberrations (NCPA) and water vapor seeing, as well as the low-wind effect. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.13527-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.13527) | **$β$ Pictoris b through the eyes of the upgraded CRIRES+**  |
|| R. Landman, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2023-11-23*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Context: High-resolution spectrographs fed by adaptive optics (AO) provide a unique opportunity to characterize directly imaged exoplanets. Observations with such instruments allow us to probe the atmospheric composition, spin rotation, and radial velocity of the planet, thereby helping to reveal information on its formation and migration history. The recent upgrade of the Cryogenic High-Resolution Infrared Echelle Spectrograph (CRIRES+) at the VLT makes it a highly suitable instrument for characterizing directly imaged exoplanets. Aims: In this work, we report on observations of $\beta$ Pictoris b with CRIRES+ and use them to constrain the planets atmospheric properties and update the estimation of its spin rotation. Methods: The data were reduced using the open-source \textit{pycrires} package. We subsequently forward-modeled the stellar, planetary, and systematic contribution to the data to detect molecules in the planet's atmosphere. We also used atmospheric retrievals to provide new constraints on its atmosphere. Results: We confidently detected water and carbon monoxide in the atmosphere of $\beta$ Pictoris b and retrieved a slightly sub-solar carbon-to-oxygen ratio, which is in agreement with previous results. The interpretation is hampered by our limited knowledge of the C/O ratio of the host star. We also obtained a much improved constraint on its spin rotation of $19.9 \pm 1.0$ km/s, which gives a rotation period of $8.7 \pm 0.8$ hours, assuming no obliquity. We find that there is a degeneracy between the metallicity and clouds, but this has minimal impact on the retrieved C/O, $v\sin{i}$, and radial velocity. Our results show that CRIRES+ is performing well and stands as a highly useful instrument for characterizing directly imaged planets. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.12932-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.12932) | **Variation of the stellar initial mass function in semi-analytical models  III: testing the cosmic ray regulated integrated galaxy-wide initial mass  function**  |
|| F. Fontanot, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2023-11-23*|
|*Comments*| *12 pages, 8 figures, 2 tables, A&A submitted, comments welcome*|
|**Abstract**| In our previous work, we derive the CR-IGIMF: a new scenario for a variable stellar initial mass function (IMF), which combines numerical results on the role played by cosmic rays in setting the thermal state of star forming gas, with the analytical approach of the integrated galaxy-wide IMF. In this work, we study the implications of this scenario for the properties of local Early-Type galaxies (ETG), as inferred from dynamical, photometric and spectroscopic studies. We implement a library of CR-IGIMF shapes in the framework of the Galaxy Evolution and Assembly (GAEA) model. Our realization includes a derivation of synthetic spectral energy distribution for each model galaxy, allowing a direct derivation of the mass fraction in the mean IMF of low-mass stars (i.e. the dwarf-to-giant ratio - $\rm f_{dg}$) and a comparison with IMF sensitive spectral features. The predictions of the GAEA model implementing the CR-IGIMF confirm our previous findings: it correctly reproduces both the observed excess of z$\sim$0 dynamical mass (mass-to-light ratios) with respect to spectroscopic (photometric) estimates assuming a universal, MW-like, IMF, and the observed increase of [$\alpha$/Fe] ratios with stellar mass in spheroidal galaxies. Moreover, this realization reproduces the increasing trends of $\rm f_{dg}$, and IMF-sensitive line-strengths with velocity dispersion, although the predicted relations are significantly shallower than the observed ones. Our results show that the CR-IGIMF is a promising scenario that reproduces at the same time dynamical, photometric and spectroscopic indications of a varying IMF in local ETGs. The shallow relations found for spectral indices suggest that either a stronger variability as a function of galaxy properties or additional dependences (e.g. as a function of star forming gas metallicity) might be required to match the strength of the observed trends. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.13388-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.13388) | **The photospheres of the hottest fastest stars in the Galaxy**  |
|| K. Werner, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2023-11-23*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| We perform nonlocal thermodynamic equilibrium (NLTE) model atmosphere analyses of the three hottest hypervelocity stars (space velocities between $\approx$ 1500-2800 km s$^{-1}$) known to date, which were recently discovered spectroscopically and identified as runaways from Type Ia supernovae. The hottest of the three (J0546$+$0836, effective temperature $T_\mathrm{eff}$ = 95,000 $\pm$ 15,000 K, surface gravity log g = $5.5 \pm 0.5$) has an oxygen-dominated atmosphere with a significant amount of carbon (C = $0.10 \pm 0.05$, O = $0.90 \pm 0.05$, mass fractions). Its mixed absorption+emission line spectrum exhibits photospheric absorption lines from O V and O VI as well as O III and O IV emission lines that are formed in a radiation-driven wind with a mass-loss rate of the order of $10^{-8}$ $M_\odot$ yr$^{-1}$. Spectroscopically, J0546$+$0836 is a [WC]-PG1159 transition-type pre-white dwarf. The second object (J0927$-$6335) is a PG1159-type white dwarf with a pure absorption-line spectrum dominated by C III/C IV and O III/O IV. We find $T_\mathrm{eff}$ = 60,000 $\pm$ 5000 K, log g = $7.0 \pm 0.5$, and a carbon- and oxygen-dominated atmosphere with C = $0.47 \pm 0.25$, O = $0.48 \pm 0.25$, and possibly a minute amount of helium (He = $0.05 \pm 0.05$). Comparison with post-AGB evolutionary tracks suggests a mass of $M\approx0.5$ $M_\odot$ for both objects, if such tracks can safely be applied to these stars. We find the third object (J1332$-$3541) to be a relatively massive ($M=0.89 M_\odot$) hydrogen-rich (DAO) white dwarf with $T_\mathrm{eff}$ = 65,657 $\pm$ 2390 K, log g = $8.38 \pm 0.08$, and abundances H = $0.65 \pm 0.04$ and He = $0.35 \pm 0.04$. We discuss our results in the context of the "dynamically driven double-degenerate double-detonation" (D$^6$) scenario proposed for the origin of these stars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2311.13097.md
    + _build/html/tmp_2311.13097/./Figure/1431_fig4_color.png
    + _build/html/tmp_2311.13097/./Figure/KB231431_gridsearch.png
    + _build/html/tmp_2311.13097/./Figure/KB231431_caustic.png
exported in  _build/html/2311.13437.md
    + _build/html/tmp_2311.13437/././Figures/C-FULLv2-CSS-LWE1.0mps.png
    + _build/html/tmp_2311.13437/././Figures/C-FULLv2-CSS-LWI1.0mps.png
    + _build/html/tmp_2311.13437/./Figures/METISblockDiagram4_handover.png
    + _build/html/tmp_2311.13437/./Figures/C-FULL-ZVDM-FKL-M4-FCN-ALPHA.png
exported in  _build/html/2311.13527.md
    + _build/html/tmp_2311.13527/./extraction_visualization4.png
    + _build/html/tmp_2311.13527/./Retrieved_parameters_robustness4.png
    + _build/html/tmp_2311.13527/./comp_with_lowres2.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thetae}{\theta_{\rm E}}$
$\newcommand{\teff}{t_{\rm eff}}$
$\newcommand{\Icat}{I_{\rm cat}}$
$\newcommand{\pie}{\pi_{\rm E}}$
$\newcommand{\pirel}{\pi_{\rm rel}}$
$\newcommand{\te}{t_{\rm E}}$
$\newcommand{\event}{KMT-2023-BLG-1431}$
$\newcommand{\an}{\theta_{*}}$
$\newcommand{\Sp}{{\it Spitzer}}$
$\newcommand{\hjd}{{\rm HJD}^{\prime}}$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand$</div>



<div id="title">

# ${\large KMT-2023-BLG-1431Lb: A New $q < 10^{-4}$ Microlensing Planet from a Subtle Signature}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.13097-b31b1b.svg)](https://arxiv.org/abs/2311.13097)<mark>Appeared on: 2023-11-23</mark> -  _PASP submitted. arXiv admin note: text overlap with arXiv:2301.06779_

</div>
<div id="authors">

A. Bell, et al. -- incl., <mark>A. Gould</mark>

</div>
<div id="abstract">

**Abstract:** The current studies of microlensing planets are limited by small number statistics. Follow-up observations of high-magnification microlensing events can efficiently form a statistical planetary sample. Since 2020, the Korea Microlensing Telescope Network (KMTNet) and the Las Cumbres Observatory (LCO) global network have been conducting a follow-up program for high-magnification KMTNet events. Here, we report the detection and analysis of a microlensing planetary event, $\event$ , for which the subtle (0.05 magnitude) and short-lived (5 hours) planetary signature was characterized by the follow-up from KMTNet and LCO. A binary-lens single-source (2L1S) analysis reveals a planet/host mass ratio of $q = (0.72 \pm 0.07) \times 10^{-4}$ , and the single-lens binary-source (1L2S) model is excluded by $\Delta\chi^2 = 80$ . A Bayesian analysis using a Galactic model yields estimates of the host star mass of $M_{\rm host} = 0.57^{+0.33}_{-0.29} M_\odot$ , the planetary mass of $M_{\rm planet} = 13.5_{-6.8}^{+8.1} M_{\earth}$ , and the lens distance of $D_{\rm L} = 6.9_{-1.7}^{+0.8}$ kpc. The projected planet-host separation of $a_\perp = 2.3_{-0.5}^{+0.5}$ au or $a_\perp = 3.2_{-0.8}^{+0.7}$ , subject to the close/wide degeneracy. We also find that without the follow-up data, the survey-only data cannot break the degeneracy of central/resonant caustics and the degeneracy of 2L1S/1L2S models, showing the importance of follow-up observations for current microlensing surveys.

</div>

<div id="div_fig1">

<img src="tmp_2311.13097/./Figure/1431_fig4_color.png" alt="Fig5" width="100%"/>

**Figure 5. -** Detailed comparison of the disfavored model fits to the anomaly: "Close Central", "Wide Central", "Close Resonant" "Wide Resonant",  and "1L2S". The top panel shows the models plotted with the data, while the middle panels show the residuals to the models. The "Close Resonant" models shows clear residuals and is ruled out. The deviations in the "Wide Resonant" and "1L2S" models are more subtle but (as shown in the bottom panel), amount to $\Delta\chi^2$ differences of $\sim 30$ and $\sim 80$, respectively, over the course of the anomaly. The "Close Central" model is competitive with the best-fit "Wide Central" model. (*fig:lc2*)

</div>
<div id="div_fig2">

<img src="tmp_2311.13097/./Figure/KB231431_gridsearch.png" alt="Fig1" width="100%"/>

**Figure 1. -** $\chi^2$ surface in the ($\log s, \log q$) plane drawn from the grid search. The upper panel displays the space that is equally divided on a ($61 \times 61$) grid with ranges of $-1.5\leq\log s \leq1.5$ and $-6.0\leq \log q \leq0$, respectively. The lower panel shows the space that is equally divided on a ($151 \times 31$) grid with ranges of $-0.15\leq\log s \leq0.15$ and $-5.0\leq \log q \leq-3.5$, respectively. Grid with $\Delta \chi^2  > 720$ are marked as blank. The labels "Close Central", "Wide Central", "Close Resonant", and "Wide Resonant" in the lower panel indicate four local minima. The two red dashed lines represent the boundaries between resonant and non-resonant caustics applying Equation (59) of [ and Dominik (1999)]().
 (*fig:grid*)

</div>
<div id="div_fig3">

<img src="tmp_2311.13097/./Figure/KB231431_caustic.png" alt="Fig2" width="100%"/>

**Figure 2. -** Geometries of the four 2L1S solutions. In each panel, the black line with an arrow represents the source trajectory with respect to the host star that is marked by blue dot, the red lines show the caustic structure, the axes are in units of the Einstein radius $\thetae$, and the magenta circle indicates the source radii.
     (*fig:cau*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.13097"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{çc}[1]{{\color{red}\textrm{#1}}}$
$\newcommand{\baselinestretch}{1.0}$
$\newcommand{\@}{maketitle}$</div>



<div id="title">

# Simulating METIS' SCAO System

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.13437-b31b1b.svg)](https://arxiv.org/abs/2311.13437)<mark>Appeared on: 2023-11-23</mark> -  _17 pages, 14 Figures, AO4ELT VII Conference - Avignon - June 2023_

</div>
<div id="authors">

<mark>M. Feldt</mark>, et al. -- incl., <mark>H. Steuer</mark>, <mark>T. Bertram</mark>

</div>
<div id="abstract">

**Abstract:** METIS, the Mid-Infrared ELT Imager and Spectrograph, is one of the four first-generation ELT instruments scheduled to see first light in 2028. Its two main science modules are supported by an adaptive optics system featuring a pyramid sensor with 90x90 subapertures working in the $H$ and $K$ bands. During the PDR and FDR phases, extensive simulations were carried out to support the sensing, reconstruction, and control concept of METIS single-conjugate adaptive optics (SCAO) system.We present details on the implementation of the COMPASS-based environment used for the simulations, the metrics used for analyzing our performance expectations, an overview of the main results, and some details on special cases like non-common path aberrations (NCPA) and water vapor seeing, as well as the low-wind effect.

</div>

<div id="div_fig1">

<img src="tmp_2311.13437/././Figures/C-FULLv2-CSS-LWE1.0mps.png" alt="Fig12.1" width="50%"/><img src="tmp_2311.13437/././Figures/C-FULLv2-CSS-LWI1.0mps.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** 
Performance and CCS response for different scalings of the 1.0 m/s low-wind case. The blue curve denotes the Strehl ratio, red denotes the maximum force required by any actuator at any time during the simulation, green shows the measured petal piston rms as usual, and the orange curve shows the maximum number of modes dropped from the command vector at any time during the simulation. In the simulation underlying the left-hand figure, mode dropping was disabled to show the strain on the maximum actuator force when the impact of the low-wind condition increases.
 (*fig:C-FULL-CCS*)

</div>
<div id="div_fig2">

<img src="tmp_2311.13437/./Figures/METISblockDiagram4_handover.png" alt="Fig1" width="100%"/>

**Figure 1. -** Simplified block diagram of the SCAO system: The Adaptive Optics Control System (AOCS) and the SCAO Module (slightly darker boxes) are the entities of the SCAO system that belong to the instrument domain. The key entities for
the real-time correction of the incoming light are located in the 'ELT' domain on the left side of the figure. In a closed wavefront control loop, the blue, near-infrared (NIR) light is used to measure the instantaneous residual
wavefront error by the WFS. The measurement signal is analyzed by the RTC, and a computed correction is sent to the Central Control System (CCS) to be applied with the M4 and ELT tip-tilt field stabilization mirror (M5) via a Local
Control System (LCS). The Focal Plane Sensor Gateway (FPS GW) provides science images to auxiliary AO loops. (*fig:SCAO-overview*)

</div>
<div id="div_fig3">

<img src="tmp_2311.13437/./Figures/C-FULL-ZVDM-FKL-M4-FCN-ALPHA.png" alt="Fig4" width="100%"/>

**Figure 4. -** Typical plot of our key performance parameters against a varying input parameter, in this case both regularization strengths of the reconstructor which were kept identical. The Strehl ratio is plotted in blue, residual tip-tilt motion of the PSF in red, residual petal piston in green, and the temporal variance contrast in orange. (*fig:stdfig_example*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.13437"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\appropto}{\mathrel{\vcenter{$
$  \offinterlineskip\halign{\hfil##\cr$
$    \propto\cr\noalign{\kern2pt}\sim\cr\noalign{\kern-2pt}}}}}$
$\newcommand{\arraystretch}{1.15}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# $\beta$ Pictoris b through the eyes of the upgraded CRIRES+

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.13527-b31b1b.svg)](https://arxiv.org/abs/2311.13527)<mark>Appeared on: 2023-11-23</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

R. Landman, et al. -- incl., <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** High-resolution spectrographs fed by adaptive optics (AO) provide a unique opportunity to characterize directly imaged exoplanets. Observations with such instruments allow us to probe the atmospheric composition, spin rotation, and radial velocity of the planet, thereby helping to reveal information on its formation and migration history. The recent upgrade of the Cryogenic High-Resolution Infrared Echelle Spectrograph (CRIRES+) at the VLT makes it a highly suitable instrument for characterizing directly imaged exoplanets. In this work, we report on observations of $\beta$ Pictoris b with CRIRES+ and use them to constrain the planets atmospheric properties and update the estimation of its spin rotation. The data were reduced using the open-source _pycrires_ package. We subsequently forward-modeled the stellar, planetary, and systematic contribution to the data to detect molecules in the planet's atmosphere. We also used atmospheric retrievals to provide new constraints on its atmosphere. We confidently detected water and carbon monoxide in the atmosphere of $\beta$ Pictoris b and retrieved a slightly sub-solar carbon-to-oxygen ratio, which is in agreement with previous results. The interpretation is hampered by our limited knowledge of the C/O ratio of the host star. We also obtained a much improved constraint on its spin rotation of $19.9 \pm 1.0$ km/s, which gives a rotation period of $8.7 \pm 0.8$ hours, assuming no obliquity. We find that there is a degeneracy between the metallicity and clouds, but this has minimal impact on the retrieved C/O, $v\sin{i}$ , and radial velocity. Our results show that CRIRES+ is performing well and stands as a highly useful instrument for characterizing directly imaged planets.

</div>

<div id="div_fig1">

<img src="tmp_2311.13527/./extraction_visualization4.png" alt="Fig1" width="100%"/>

**Figure 1. -** Visualization of the data extraction pipeline. Top row shows the product of the _obs\_nodding_ recipe from the ESO CRIRES+ data reduction pipeline. The middle row shows the rectified 2D spectrum corrected for the slit tilt and curvature using _pycrires_ and the bottom row shows the spectrum at the location of the planet, plotted together with a telluric model generated with SkyCalc. (*fig:data_reduction*)

</div>
<div id="div_fig2">

<img src="tmp_2311.13527/./Retrieved_parameters_robustness4.png" alt="Fig3" width="100%"/>

**Figure 3. -** Constraints on a subset of the parameters from the atmospheric retrievals using the models discussed in Section \ref{sec:atmosphere_models}. The RV shown here is in the barycentric restframe. (*fig:retrieved_params*)

</div>
<div id="div_fig3">

<img src="tmp_2311.13527/./comp_with_lowres2.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison of the spectral energy distributions of our retrieved models compared to the low-resolution GPI data from [Chilcote and Pueyo (2017)]() and GRAVITY data from [Nowak and Lacour (2020)](). Since we do not retrieve radius, the flux of our models is scaled such that it matches the GPI flux in the K-band. The solid line shows the median value and the colored region shows the 16th and 84th percentiles. (*fig:comp_lowres*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.13527"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

327  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
